In [101]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from sasdata.dataloader.loader import Loader 
from bumps.names import FitProblem, FreeVariables
from sasmodels.core import load_model
from sasmodels.bumps_model import Model, Experiment

In [102]:
# Following are most likely spherical micelles with a PHFBA core and PDEGEEA corona
TESTING = True 
SLD_CORE = 1.85
SLD_CORONA = 0.817
SLD_SOLVENT_LIST = {'dTHF': 6.349, 'THF': 0.183, 'D2O':6.36, 
'H2O':-0.561, 'dCF': 3.156, 'dTol':5.664, 'dAcetone':5.389,
'dTHF0':6.360, 'dTHF25':6.357, 'dTHF50':6.355, 'dTHF75':6.352,'hTHF':1.0
}
NUM_STEPS = 5 if TESTING else 5e2

SI_FILE_LOC = './EXPTDATA_V2/sample_info_OMIECS.csv'
DATA_DIR = './EXPTDATA_V2/inco_bg_sub/'

In [103]:
def load_data_from_file(fname, use_trim=False):
    SI = pd.read_csv(SI_FILE_LOC)
    flag = SI["Filename"]==fname
    metadata = SI[flag]
    loader = Loader()
    data = loader.load(DATA_DIR+'%s'%fname)[0]
    
    if not use_trim:
        data.qmin = min(data.x)
        data.qmax = max(data.x)
    else:
        data.qmin = min(data.x)
        data.qmax = data.x[-metadata['Highq_trim']]
        
    return data, metadata

In [104]:
FIT_KEYS = [116,118,129,125,127,132,134,135,136,138,139,140,931,932,933,964,965,970,971] 
BLOCK_KEYS = [('DEG', '25b'), ('DEG', '50'), ('DEG', '75'), ('PEG', '25'), ('PEG', '50')]
FIT_BLOCK_KEY = ('DEG', '25b') 
SIMUL_FILENAMES = [] 
SI = pd.read_csv(SI_FILE_LOC)
for key, values in SI.iterrows():
        if values['Sample'] in FIT_KEYS:
                if (values['EG_group']==FIT_BLOCK_KEY[0] and values['Flor_block']==FIT_BLOCK_KEY[1]):
                    fname = values['Filename']
                    SIMUL_FILENAMES.append(fname)
                    print(fname,values['EG_group'], values['Flor_block'])


D50F25b_10dTol.sub DEG 25b
D50F25b_10dTHF75.sub DEG 25b


In [105]:
datasets = [load_data_from_file(f)[0] for f in SIMUL_FILENAMES]

Traceback (most recent call last):
  File "/Users/pozzolabadmin/opt/anaconda3/envs/micelles/lib/python3.11/site-packages/sasdata/dataloader/loader.py", line 79, in load
    data_list = super().load(path, ext=ext)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pozzolabadmin/opt/anaconda3/envs/micelles/lib/python3.11/site-packages/sasdata/data_util/registry.py", line 119, in load
    raise NoKnownLoaderException("No loaders match extension in %r"
sasdata.data_util.loader_exceptions.NoKnownLoaderException: No loaders match extension in './EXPTDATA_V2/inco_bg_sub/D50F25b_10dTol.sub'
Traceback (most recent call last):
  File "/Users/pozzolabadmin/opt/anaconda3/envs/micelles/lib/python3.11/site-packages/sasdata/dataloader/loader.py", line 79, in load
    data_list = super().load(path, ext=ext)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pozzolabadmin/opt/anaconda3/envs/micelles/lib/python3.11/site-packages/sasdata/data_util/registry.py", line 119, in load
    raise

In [106]:
def setup_model(model):
    if model=='sph':
        sas_model = load_model("../models/spherical_micelle.py")
        bumps_model = Model(model=sas_model)

    elif model=='cyl':
        sas_model = load_model("../models/cylindrical_micelle.py")
        bumps_model = Model(model=sas_model)
        bumps_model.length_core.range(20.0,1000.0)

    elif model=='elp':
        sas_model = load_model("../models/ellipsoidal_micelle.py")
        bumps_model = Model(model=sas_model)
        bumps_model.eps.range(1.0,20.0)

    bumps_model.d_penetration.range(0.75, 1.0)    
    bumps_model.scale.range(1e-15, 1e-5)
    # use default bounds
    bumps_model.v_core.fixed = False 
    bumps_model.v_corona.fixed = False
    bumps_model.n_aggreg.fixed = True
    # use fixed values
    bumps_model.background.fixed = True 
    bumps_model.background.value = 0.0
    bumps_model.sld_core.fixed = True 
    bumps_model.sld_core.value = SLD_CORE
    bumps_model.sld_corona.fixed = True 
    bumps_model.sld_corona.value = SLD_CORONA
    bumps_model.sld_solvent.fixed = True 

    return sas_model, bumps_model


In [107]:
sas_model, bumps_model = setup_model('sph')

In [108]:
free = FreeVariables(names=['data_%d'%i for i in range(len(datasets))],
                     radius_core = bumps_model.radius_core,
                     radius_core_pd = bumps_model.radius_core_pd,
                     rg = bumps_model.rg,
                     )
free.radius_core.range(20.0, 200.0)
free.radius_core_pd.range(0.0, 0.5)
free.rg.range(0.0, 200.0)

In [109]:
# Setup the experiments, sharing the same model across all datasets.
M = [Experiment(data=data, model=bumps_model, name='data_%d'%i) for i,data in enumerate(datasets)]
problem = FitProblem(M, freevars=free)

In [110]:
from bumps.fitters import *  

driver = FitDriver(fitclass=DEFit, problem=problem, mapper=None, steps=NUM_STEPS)
driver.clip() # make sure fit starts within domain
x0 = problem.getp()
x, fx = driver.fit()
problem.setp(x)
dx = driver.stderr()
print("final chisq", problem.chisq_str())
driver.show_err() 
print('Final fitting parameters for : ', fname)
print('Parameter Name\tFitted value')
for key, param in bumps_model.parameters().items():
    if not param.fixed:
        print(key, '\t', '%.2e'%param.value)

step 1 cost 1584.423(20)
                      data_0 radius_core .|........    44.8813 in (20,200)
                      data_1 radius_core .....|....    118.283 in (20,200)
                   data_0 radius_core_pd .......|..   0.361853 in (0,0.5)
                   data_1 radius_core_pd .|........  0.0875367 in (0,0.5)
                               data_0 rg .........|    181.512 in (0,200)
                               data_1 rg ......|...    139.186 in (0,200)
                           d_penetration ....|.....   0.856772 in (0.75,1)
                                   scale ......|... 6.77306e-06 in (1e-15,1e-05)
                                  v_core .......|..    8068.39 in (0,inf)
                                v_corona .......|..    6172.76 in (0,inf)
step 2 cost 1584.423(20)
step 3 cost 1584.423(20)
step 4 cost 1584.423(20)
final chisq 1584.423(20)
=== Uncertainty from curvature:     name   value(unc.) ===
                      data_0 radius_core   44.881(59)     
       

In [114]:
for model in problem.model_parameters()["models"]:
    for name, param in model.items():
        if not param.fixed:
            print(name, param.value)

scale 6.773063095337969e-06
v_core 8068.39238120536
v_corona 6172.756368640786
d_penetration 0.8567717211734434
scale 6.773063095337969e-06
v_core 8068.39238120536
v_corona 6172.756368640786
d_penetration 0.8567717211734434


In [123]:
for name, param in problem.model_parameters()["freevars"].items():
    for p in param:
        print(p.name, p.value)

data_0 radius_core 44.881309283157215
data_1 radius_core 118.28329049315936
data_0 radius_core_pd 0.3618533688553785
data_1 radius_core_pd 0.08753671811856878
data_0 rg 181.5116347607419
data_1 rg 139.1862251855417
